In [1]:
# Import appropriate libraries

import pyodbc
import os.path
import glob
import pandas as pd
import numpy as np
from datetime import date
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta

import warnings
warnings.simplefilter("ignore")

pd.set_option('display.max_columns', None)

In [2]:
# SQL python connection to our server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=sql-2-db;'
                      'Database=CBQ2;')
                    
cursor = conn.cursor()

In [3]:
query_item = '''
SELECT
    i.ISBN
    ,i.SHORT_TITLE title
    ,i.PUBLISHER_CODE publisher
    ,case                             
        when i.PUBLISHER_CODE = 'Tourbillon' then 'TW'                      
        when i.PUBLISHER_CODE = 'Sierra Club' then 'SC'                     
        when i.PUBLISHER_CODE IN('Glam Media','Benefit','PQ Blackwell','San Francisco Art Institute','AFO LLC','FareArts','Sager') then 'CD'                 
        when i.PUBLISHER_CODE = 'Creative Company' then 'CC'   
        when i.PUBLISHER_CODE = 'Do Books' then 'DO'
        when i.PUBLISHER_CODE = 'Levine Querido' then 'LQ'
        when i.PUBLISHER_CODE = 'AMMO Books' then 'AM'                                           
        when i.PUBLISHING_GROUP = 'GAL' then 'GA'                                                      
        when i.PUBLISHING_GROUP = 'GAL-CL' then 'CL'                        
        when i.PUBLISHING_GROUP = 'MUD' then 'MP'
        when i.PUBLISHING_GROUP = 'GAL-BM' then 'BM'             
        when i.PUBLISHING_GROUP in('LAU-BIS') then 'LKBS'                          
        when i.PUBLISHER_CODE = 'Laurence King' and i.PRODUCT_TYPE = 'FT' then 'LKGI'                      
        when i.PUBLISHER_CODE = 'Laurence King' and i.PRODUCT_TYPE <> 'FT' then 'LKBK'         
        when i.PUBLISHER_CODE = 'Hardie Grant Publishing' then 'HG'  
        when i.PUBLISHING_GROUP in('BAR-ART','BAR-ENT','BAR-LIF') then 'BAR'                  
        else i.PUBLISHING_GROUP                 
    end pgrp
FROM                
     ebs.Item i
WHERE
    i.PRODUCT_TYPE in('BK','FT','DI')
    --AND i.AVAILABILITY_STATUS not in('OP','WIT','OPR','NOP','OSI','PC','DIS','CS','POS')
    AND i.AVAILABILITY_STATUS is not null

'''

In [4]:
df_item = pd.read_sql_query(query_item,conn)

In [5]:
# Location of DOWNLOAD FOLDER
folder_path = r'G:\SALES\Amazon\RBH\DOWNLOADED_FILES'

file_glob_catalog = r'\*Catalog*csv'
files = glob.glob(folder_path + file_glob_catalog)
file_catalog = max(files, key=os.path.getctime)

file_glob_inventory = r'\*inventory*csv'
files = glob.glob(folder_path + file_glob_inventory)
file_inventory = max(files, key=os.path.getctime)

print()
print(f"The latest catalog file:\n{file_catalog}")
print()
print(f"The latest inventory file:\n{file_inventory}")
print()


The latest catalog file:
G:\SALES\Amazon\RBH\DOWNLOADED_FILES\Catalog_Manufacturing_UnitedStates.csv

The latest inventory file:
G:\SALES\Amazon\RBH\DOWNLOADED_FILES\Inventory_Manufacturing_Retail_UnitedStates_Daily_8-10-2024_8-10-2024.csv



# Read in Catalog File

In [6]:
# Define a custom date parsing function
date_parser = lambda x: pd.to_datetime(x, format='%Y-%m-%d %H:%M:%S', errors='coerce')

cols_cat = ['ASIN','EAN','ISBN-13','Model Number','Release Date']

df_catalog = pd.read_csv(file_catalog
                        ,skiprows = 1
                        ,na_values = '—'
                        ,usecols = cols_cat
                        ,parse_dates= ['Release Date']
#                         ,date_parser=date_parser
                        )

# Remove duplicate rows based on a specific column
df_catalog.drop_duplicates(subset='ASIN', keep='first', inplace=True)

df_catalog.shape

(29187, 5)

In [7]:
df_catalog.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29187 entries, 0 to 29224
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ASIN          29187 non-null  object        
 1   EAN           29154 non-null  object        
 2   ISBN-13       29154 non-null  object        
 3   Model Number  6752 non-null   object        
 4   Release Date  29187 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(4)
memory usage: 1.3+ MB


In [8]:
df_catalog.loc[df_catalog.ASIN == '1761213105']

,ASIN,EAN,ISBN-13,Model Number,Release Date
7887,1761213105,9781761213106,9781761213106,NaN,2024-04-02


# Read in Amazon Inventory File

In [9]:
# Inventory file to Dataframe
cols_invt = ['ASIN','Unfilled Customer Ordered Units']

df_inventory = pd.read_csv(file_inventory
                        ,skiprows = 1
                        ,na_values = '—'
                        ,usecols = cols_invt
                        ,infer_datetime_format=False
                          )

# df_inventory['Unfilled Customer Ordered Units'] = pd.to_numeric(df_inventory['Unfilled Customer Ordered Units']\
#                                                 .str.replace(',', ''), errors='coerce')
#                                                 .fillna(0)
#                                                 .astype('Int64')

df_inventory.shape

(12465, 2)

In [10]:
df_inventory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12465 entries, 0 to 12464
Data columns (total 2 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   ASIN                             12465 non-null  object
 1   Unfilled Customer Ordered Units  6230 non-null   object
dtypes: object(2)
memory usage: 194.9+ KB


In [11]:
df_inventory.loc[df_inventory.ASIN == '1761213105']

,ASIN,Unfilled Customer Ordered Units
10767,1761213105,NaN


In [12]:
len(df_inventory.ASIN.unique())

12465

In [13]:
df_inventory['Unfilled Customer Ordered Units']

0         52
1         19
2         64
3        140
4        352
        ... 
12460      0
12461      1
12462      0
12463      2
12464      3
Name: Unfilled Customer Ordered Units, Length: 12465, dtype: object

In [14]:
df_inventory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12465 entries, 0 to 12464
Data columns (total 2 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   ASIN                             12465 non-null  object
 1   Unfilled Customer Ordered Units  6230 non-null   object
dtypes: object(2)
memory usage: 194.9+ KB


### Figuring out Dups!

In [15]:
df_inventory.loc[df_inventory.ASIN == '1797229176']

,ASIN,Unfilled Customer Ordered Units
9622,1797229176,419


In [16]:
df_catalog.loc[df_catalog.ASIN == '1797229176']

,ASIN,EAN,ISBN-13,Model Number,Release Date
18047,1797229176,9781797229171,9781797229171,NaN,2024-12-03


# Merging Files

In [17]:
df_nyp = pd.merge(df_inventory,df_catalog,on = 'ASIN')

df_nyp.shape

(12457, 6)

In [18]:
df_nyp.loc[df_nyp.ASIN == '1797229176']

,ASIN,Unfilled Customer Ordered Units,EAN,ISBN-13,Model Number,Release Date
9616,1797229176,419,9781797229171,9781797229171,NaN,2024-12-03


In [19]:
# Correct list of ISBN
isbn_list = df_item.ISBN.unique()

# Create the "ISBN" column using NumPy's where function
df_nyp['ISBN'] = np.where(df_nyp['EAN'].isin(isbn_list), df_nyp['EAN'],
                          np.where(df_nyp['ISBN-13'].isin(isbn_list), df_nyp['ISBN-13'],
                                   df_nyp['Model Number']))

# Drop the unnecessary columns
df_nyp.drop(['EAN', 'ISBN-13', 'Model Number'], axis=1, inplace=True)

df_nyp = df_nyp.dropna(subset = ['Unfilled Customer Ordered Units'])

df_nyp.dropna(subset=['Unfilled Customer Ordered Units'], inplace=True)

# Check the shape of the DataFrame
print(df_nyp.shape)

(6223, 4)


In [20]:
df_item.head()

,ISBN,title,publisher,pgrp
0,9781452103327,Stoner Coffee Table Book,Chronicle,ENT
1,9781452110462,Stoner Coffee Table Book (ebook),Chronicle,ENT
2,9780811868693,Stonewall Kitchen: Appetizers hc *OSI*,Chronicle,FWN
3,9780811868679,Stonewall Kitchen Breakfast hc,Chronicle,FWN
4,9780811868709,Stonewall Kitchen Grilling *OSI*,Chronicle,FWN


In [21]:
df_catalog

,ASIN,EAN,ISBN-13,Model Number,Release Date
0,168152645X,9781681526454,9781681526454,NaN,2021-01-05
1,9063694695,9789063694692,9789063694692,NaN,2018-01-30
2,1787139980,9781787139985,9781787139985,NaN,2024-03-19
3,1878271083,9781878271082,9781878271082,NaN,1997-12-01
4,0811853810,9780811853811,9780811853811,NaN,2006-07-20
...,...,...,...,...,...
29220,0735376522,9780735376526,0735376522,9780735376526,2023-01-17
29221,0735370125,9780735370128,0735370125,9780735370128,2021-11-09
29222,0735365822,9780735365827,0735365822,9780735365827,2020-06-30
29223,0735362017,9780735362017,0735362017,9780735362017,2020-08-18


In [22]:
df_nyp = pd.merge(df_nyp,df_item.loc[:,['ISBN','title','publisher']]
                  ,on='ISBN'
                 ,how='left')

df_nyp = df_nyp.sort_values(by='Unfilled Customer Ordered Units', ascending=False)

In [23]:
df_nyp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6895 entries, 3999 to 1043
Data columns (total 6 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   ASIN                             6895 non-null   object        
 1   Unfilled Customer Ordered Units  6895 non-null   object        
 2   Release Date                     6895 non-null   datetime64[ns]
 3   ISBN                             6167 non-null   object        
 4   title                            6848 non-null   object        
 5   publisher                        6848 non-null   object        
dtypes: datetime64[ns](1), object(5)
memory usage: 377.1+ KB


In [24]:
df_nyp['Release Date'].value_counts()

2024-08-13    310
2023-01-17    114
2024-09-03     97
2023-01-01     71
2024-03-05     60
             ... 
2015-07-09      1
2018-05-24      1
2005-02-17      1
2009-10-14      1
2018-09-02      1
Name: Release Date, Length: 918, dtype: int64

In [25]:
df_nyp = df_nyp[['ASIN','ISBN','title','publisher','Release Date','Unfilled Customer Ordered Units']]

df_nyp_sorted = df_nyp.sort_values(by='Unfilled Customer Ordered Units', ascending=False).reset_index(drop=True)

In [26]:
# Remove titles that are not nyp
df_nyp_sorted = df_nyp_sorted[df_nyp_sorted['Release Date'] >= pd.to_datetime(date.today())]

# Change the name of "Unfilled Customer Ordered Units" to "nyp"
df_nyp_sorted.rename(columns={'Unfilled Customer Ordered Units': 'Orders'}, inplace=True)

In [27]:
df_nyp_sorted['Orders'] = df_nyp_sorted['Orders'].str.replace(',', '').astype('float32')
df_nyp_sorted['Orders'] = df_nyp_sorted['Orders'].astype('float32')

In [28]:
df_nyp_sorted.groupby('publisher')['Orders'].sum().sort_values(ascending=False).reset_index()

,publisher,Orders
0,Chronicle,8948.0
1,Hardie Grant Publishing,2378.0
2,Galison,1123.0
3,Laurence King,1051.0
4,Levine Querido,184.0
5,Paperblanks,128.0
6,Creative Company,96.0
7,?,9.0
8,Princeton,9.0
9,Tourbillon,0.0


In [29]:
def convert_to_float(x):
    if isinstance(x, str):
        return float(x.replace(',', ''))
    else:
        return x

df_nyp_sorted['Orders'] = df_nyp_sorted['Orders'].apply(convert_to_float)


# Sort DataFrame by the "Orders" column in descending order
df_nyp_sorted = df_nyp_sorted.sort_values(by='Orders', ascending=False)

In [30]:
# # Main Folder
# folder = r'G:\SALES\Amazon\PREORDERS\2024\\'

# # Get the current date
# current_date = date.today().strftime('%Y-%m-%d')

# # Save DataFrame as an Excel file
# file_name = f'preorders_{current_date}.xlsx'

# path = folder + file_name

# writer = pd.ExcelWriter(path, engine='xlsxwriter')

# df_nyp_sorted.to_excel(writer, sheet_name='nyp', index=False)

# filter_chronicle = df_nyp_sorted.publisher=='Chronicle'

# df_nyp_sorted.loc[filter_chronicle].to_excel(writer, sheet_name='nyp_cb', index=False)
# df_nyp_sorted.loc[~filter_chronicle].to_excel(writer, sheet_name='nyp_dp', index=False)

# writer.save()
# writer.close()

In [31]:
# Main Folder
folder = fr'G:\SALES\Amazon\PREORDERS\2024\\'

# Get the current date
current_date = date.today().strftime('%Y-%m-%d')

# File names
dated_file_name = f'preorders_{current_date}.xlsx'
current_file_name = 'current_amaz_preorders.xlsx'

# Paths
dated_path = folder + dated_file_name
current_path = folder + current_file_name

# Create Excel writers
dated_writer = pd.ExcelWriter(dated_path, engine='xlsxwriter')
current_writer = pd.ExcelWriter(current_path, engine='xlsxwriter')

# Save DataFrame to the dated file
df_nyp_sorted.to_excel(dated_writer, sheet_name='nyp', index=False)
df_nyp_sorted.loc[df_nyp_sorted.publisher == 'Chronicle'].to_excel(dated_writer, sheet_name='nyp_cb', index=False)
df_nyp_sorted.loc[df_nyp_sorted.publisher != 'Chronicle'].to_excel(dated_writer, sheet_name='nyp_dp', index=False)

# Save DataFrame to the current file
df_nyp_sorted.to_excel(current_writer, sheet_name='nyp', index=False)
df_nyp_sorted.loc[df_nyp_sorted.publisher == 'Chronicle'].to_excel(current_writer, sheet_name='nyp_cb', index=False)
df_nyp_sorted.loc[df_nyp_sorted.publisher != 'Chronicle'].to_excel(current_writer, sheet_name='nyp_dp', index=False)

# Save and close the writers
dated_writer.save()
dated_writer.close()

current_writer.save()
current_writer.close()